# Advanced scikit-learn

Authors: [Alexandre Gramfort](http://alexandre.gramfort.net), [Thomas Moreau](https://tommoral.github.io/about.html), and [Pedro L. C. Rodrigues](https://plcrodrigues.github.io/).

<img src="figures/sklearn-logo.png" width="150">

The aim of this notebook is:

  - To explain the **full scikit-learn API** (estimators, transformers, classifiers, regressors, splitters). 
      - A good reference is [here](https://scikit-learn.org/1.5/developers/develop.html) too.
  - To explain how to assemble these objects in complex **pipelines with mixed data types** (numerical, categorical, etc.) using `Pipeline` and `ColumnTransformer` objects.
  - Have you **write your own** transformer, splitter and classifier.
  
## Table of contents

* [1 Working only with numerical data](#workingnumerical)
    * [1.1 Pandas preprocessing](#workingnumerical_pandas)
    * [1.2 Making it less error prone using scikit-learn](#workingnumerical_errorprone)    
* [2 Working only with categorical data](#workingcategorical)
* [3 Combining both categorical and numerical data in the pipeline](#combining)
* [4 From one split to cross-validation](#crossvalidation)

To explain these concepts we will start from a full working code based on the Titanic dataset. 

Then, we will deconstruct all the blocks and start writing our own Python classes.

First, let's fetch the Titanic dataset directly from [OpenML](https://openml.org/).

In [347]:
import pandas as pd
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt

In [323]:
X_df, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X_df.head()

/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


The classification task is to predict whether or not a person will survive the Titanic disaster.

In [4]:
y

0       1
1       1
2       0
3       0
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: survived, Length: 1309, dtype: category
Categories (2, object): ['0', '1']

We will split the data into a training and a testing set.

In [150]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, random_state=42
)

In [151]:
y_train.sort_index()

0       1
1       1
2       0
3       0
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: survived, Length: 981, dtype: category
Categories (2, object): ['0', '1']

In [152]:
y_test.sort_index()

10      0
23      1
29      1
31      1
32      1
       ..
1293    0
1295    0
1297    0
1298    0
1301    0
Name: survived, Length: 328, dtype: category
Categories (2, object): ['0', '1']

<div class="alert alert-success">
    <p><b>QUESTIONS</b>:</p>
    <ul>
        <li>What would happen if you tried to fit a <tt>LogisticRegression</tt> classifier?</li>
    </ul>
</div>

In [158]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# TODO
# model.fit(X_train, y_train)
# The error occurs because LogisticRegression in sklearn cannot handle non-numeric (string) data
# The training data X_train contains string values. For ML models like LogisitcRegression, all input features need to be numerical

## To fix this issue, we need to preprocess the data to convert string features into a numerical representation that the model can understand.

# 1 Working only with numerical data <a class="anchor" id="workingnumerical"></a> [↑](#Table-of-contents)

Let's start with a model using only numerical columns.

In [159]:
X_df.dtypes

pclass        float64
name           object
sex          category
age           float64
sibsp         float64
parch         float64
ticket         object
fare          float64
cabin          object
embarked     category
boat           object
body          float64
home.dest      object
dtype: object

In [169]:
# select only numerical columns
X_num = X_df.select_dtypes(exclude=["object", "category"])
X_num.columns

Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'body'], dtype='object')

In [173]:
# x_num_columuns = list(X_num.columns)
x_num_columuns = X_num.columns.tolist()
x_num_columuns

['pclass', 'age', 'sibsp', 'parch', 'fare', 'body']

In [164]:
help(X_df.select_dtypes)

Help on method select_dtypes in module pandas.core.frame:

select_dtypes(include=None, exclude=None) -> 'DataFrame' method of pandas.core.frame.DataFrame instance
    Return a subset of the DataFrame's columns based on the column dtypes.
    
    Parameters
    ----------
    include, exclude : scalar or list-like
        A selection of dtypes or strings to be included/excluded. At least
        one of these parameters must be supplied.
    
    Returns
    -------
    DataFrame
        The subset of the frame including the dtypes in ``include`` and
        excluding the dtypes in ``exclude``.
    
    Raises
    ------
    ValueError
        * If both of ``include`` and ``exclude`` are empty
        * If ``include`` and ``exclude`` have overlapping elements
        * If any kind of string dtype is passed in.
    
    See Also
    --------
    DataFrame.dtypes: Return Series with the data type of each column.
    
    Notes
    -----
    * To select all *numeric* types, use ``np.number

## 1.1 Pandas preprocessing  <a class="anchor" id="workingnumerical_pandas"></a> [↑](#Table-of-contents)

Before using scikit-learn, we will do some simple preprocessing using pandas. First, let's select only a few the numerical columns:

In [178]:
num_cols = ['pclass', 'age', 'parch', 'fare']

X_train_num = X_train[num_cols]
X_test_num = X_test[num_cols]
X_train_num

,pclass,age,parch,fare
1139,3.0,38.0,0.0,7.8958
678,3.0,6.0,1.0,15.2458
290,1.0,52.0,1.0,79.6500
285,1.0,67.0,0.0,221.7792
1157,3.0,18.0,1.0,20.2125
...,...,...,...,...
1095,3.0,NaN,0.0,7.6292
1130,3.0,18.0,0.0,7.7750
1294,3.0,28.5,0.0,16.1000
860,3.0,26.0,0.0,7.9250


<div class="alert alert-success">
    <p><b>QUESTIONS</b>:</p>
    <ul>
        <li>And now, what would happen if you tried to fit a <tt>LogisticRegression</tt> classifier ?</li>
    </ul>
</div>

In [184]:
model = LogisticRegression()
# TODO
# model.fit(X_train_num, y_train)
# still not working to fit this LogisticRegression model because there are missing values in some columns

We might want to look into a summary of the data that we try to fit.

In [185]:
X_train_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 1139 to 1126
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  981 non-null    float64
 1   age     784 non-null    float64
 2   parch   981 non-null    float64
 3   fare    980 non-null    float64
dtypes: float64(4)
memory usage: 38.3 KB


Since there are some missing data, we can replace them with a mean.

In [186]:
X_train_num_imputed = X_train_num.fillna(X_train_num.mean())
X_train_num_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 1139 to 1126
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  981 non-null    float64
 1   age     981 non-null    float64
 2   parch   981 non-null    float64
 3   fare    981 non-null    float64
dtypes: float64(4)
memory usage: 38.3 KB


In [187]:
model.fit(X_train_num_imputed, y_train)

LogisticRegression()

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
    <li>What should we do if there are also missing values in the test set?</li>
    <li>Process the test set so as to be able to compute the test score of the model.</li>
    </ul>
</div>

Solution is in `solutions/01-pandas_fillna_test.py`

In [189]:
# TODO
X_test_num.info()       ## there are missing values in the test data, for the 'age' column

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 1148 to 533
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  328 non-null    float64
 1   age     262 non-null    float64
 2   parch   328 non-null    float64
 3   fare    328 non-null    float64
dtypes: float64(4)
memory usage: 12.8 KB


In [192]:
X_test_num_imputed = X_test_num.fillna(X_train_num.mean())
# X_test_num_imputed.info()   ## no missing values in the test data
model.score(X_test_num_imputed, y_test)

0.6707317073170732

## 1.2 Making it less error prone using scikit-learn <a class="anchor" id="workingnumerical_errorprone"></a> [↑](#Table-of-contents)

Scikit-learn provides several useful **transformers** to preprocess data. Its basic API is illustrated below:

<img src="figures/sklearn-transformers-api.png" width="450">

Basically, a transformed is used in three sequential steps:

1. Fit the transformer to the training data
2. Transform the training data
3. Transform the test data

Some examples of transformers are [scalers](https://scikit-learn.org/1.5/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling), [normalizers](https://scikit-learn.org/1.5/modules/preprocessing.html#normalization), and [categorical encoders](https://scikit-learn.org/1.5/modules/preprocessing.html#encoding-categorical-features). 

The `sklearn.impute.SimpleImputer` is a transformer allowing for the same job than the processing done with Pandas above. As we will see later, it integrates nicely with other scikit-learn components when using pipelines.

In [195]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="mean")

In [196]:
SimpleImputer?      #help(SimpleImputer)

Init signature:
SimpleImputer(
    *,
    missing_values=nan,
    strategy='mean',
    fill_value=None,
    copy=True,
    add_indicator=False,
    keep_empty_features=False,
)
Docstring:     
Univariate imputer for completing missing values with simple strategies.

Replace missing values using a descriptive statistic (e.g. mean, median, or
most frequent) along each column, or using a constant value.

Read more in the :ref:`User Guide <impute>`.

.. versionadded:: 0.20
   `SimpleImputer` replaces the previous `sklearn.preprocessing.Imputer`
   estimator which is now removed.

Parameters
----------
missing_values : int, float, str, np.nan, None or pandas.NA, default=np.nan
    The placeholder for the missing values. All occurrences of
    `missing_values` will be imputed. For pandas' dataframes with
    nullable integer dtypes with missing values, `missing_values`
    can be set to either `np.nan` or `pd.NA`.

strategy : str, default='mean'
    The imputation strategy.

    - If "mean",

As any estimator in scikit-learn, a transformer has a `fit` method which should be called on the training data to learn the required statistics. In the case of a mean imputer, we need to compute the mean for each feature.

In [198]:
imputer.fit(X_train_num)

SimpleImputer()

In [234]:
imputer.statistics_

array([ 2.29867482, 29.34768278,  0.39143731, 33.68646633])

To impute the values by the mean, we can use the `transform` method.

In [202]:
imputer.transform(X_train_num)

array([[ 3.    , 38.    ,  0.    ,  7.8958],
       [ 3.    ,  6.    ,  1.    , 15.2458],
       [ 1.    , 52.    ,  1.    , 79.65  ],
       ...,
       [ 3.    , 28.5   ,  0.    , 16.1   ],
       [ 3.    , 26.    ,  0.    ,  7.925 ],
       [ 3.    , 28.    ,  0.    ,  7.8958]])

In [213]:
test = pd.DataFrame(imputer.transform(X_train_num), columns=X_train_num.columns, index=X_train_num.index)

In [224]:
X_train_num.sort_index(inplace=True)
X_train_num[X_train_num['age'].isna() == True]

,pclass,age,parch,fare
15,1.0,NaN,0.0,25.9250
37,1.0,NaN,0.0,26.5500
40,1.0,NaN,0.0,39.6000
46,1.0,NaN,0.0,31.0000
59,1.0,NaN,0.0,27.7208
...,...,...,...,...
1284,3.0,NaN,0.0,8.0500
1291,3.0,NaN,0.0,8.7125
1302,3.0,NaN,0.0,7.2250
1303,3.0,NaN,0.0,14.4583


As previoulsy mentioned, we should always use the values estimated using the training set in `fit` when imputing the test set. Otherwise, we risk suffering from overfitting due to [data leakage](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage).

<div class="alert alert-warning">
<b>What is a "Transformer"?</b>: <br/>

Every transform in `sklearn` implements at least the following methods:

<ul>
    <li><tt>fit(X, y=None)</tt></li>
    <li><tt>transform(X)</tt></li>
    <li><tt>get_params()</tt></li>
    <li><tt>set_params(**kwargs)</tt></li>  
</ul>
</div>

In [19]:
params = imputer.get_params()
params

{'add_indicator': False,
 'copy': True,
 'fill_value': None,
 'keep_empty_features': False,
 'missing_values': nan,
 'strategy': 'mean'}

In [20]:
imputer.fit?

Signature: imputer.fit(X, y=None)
Docstring:
Fit the imputer on `X`.

Parameters
----------
X : {array-like, sparse matrix}, shape (n_samples, n_features)
    Input data, where `n_samples` is the number of samples and
    `n_features` is the number of features.

y : Ignored
    Not used, present here for API consistency by convention.

Returns
-------
self : object
    Fitted estimator.
File:      ~/opt/anaconda3/lib/python3.9/site-packages/sklearn/impute/_base.py
Type:      method

In [21]:
imputer.transform?

Signature: imputer.transform(X)
Docstring:
Impute all missing values in `X`.

Parameters
----------
X : {array-like, sparse matrix}, shape (n_samples, n_features)
    The input data to complete.

Returns
-------
X_imputed : {ndarray, sparse matrix} of shape                 (n_samples, n_features_out)
    `X` with imputed values.
File:      ~/opt/anaconda3/lib/python3.9/site-packages/sklearn/impute/_base.py
Type:      method

Let's look at the attributes of our `imputer`

In [22]:
public_attributes = [attr for attr in dir(imputer) if not attr.startswith('_')]
public_attributes

['add_indicator',
 'copy',
 'feature_names_in_',
 'fill_value',
 'fit',
 'fit_transform',
 'get_feature_names_out',
 'get_metadata_routing',
 'get_params',
 'indicator_',
 'inverse_transform',
 'keep_empty_features',
 'missing_values',
 'n_features_in_',
 'set_output',
 'set_params',
 'statistics_',
 'strategy',
 'transform']

We have among these attributes:

- **parameters** (keys in get_params method output)
- **methods** (fit, transform, etc.)
- **estimated quantities** that appear after a `fit` (ending with `_`)

In [23]:
public_methods = [
    attr for attr in dir(imputer)
    if not attr.startswith('_') and
    not attr.endswith('_') and
    attr not in params]
public_methods

['fit',
 'fit_transform',
 'get_feature_names_out',
 'get_metadata_routing',
 'get_params',
 'inverse_transform',
 'set_output',
 'set_params',
 'transform']

In [24]:
imputer.inverse_transform?

Signature: imputer.inverse_transform(X)
Docstring:
Convert the data back to the original representation.

Inverts the `transform` operation performed on an array.
This operation can only be performed after :class:`SimpleImputer` is
instantiated with `add_indicator=True`.

Note that `inverse_transform` can only invert the transform in
features that have binary indicators for missing values. If a feature
has no missing values at `fit` time, the feature won't have a binary
indicator, and the imputation done at `transform` time won't be
inverted.

.. versionadded:: 0.24

Parameters
----------
X : array-like of shape                 (n_samples, n_features + n_features_missing_indicator)
    The imputed data to be reverted to original data. It has to be
    an augmented array of imputed data and the missing indicator mask.

Returns
-------
X_original : ndarray of shape (n_samples, n_features)
    The original `X` with missing values as it was prior
    to imputation.
File:      ~/opt/anacond

Estimated quantities:

In [25]:
fit_attributes = [
    attr for attr in dir(imputer)
    if not attr.startswith('_') and
    attr.endswith('_')]
fit_attributes

['feature_names_in_', 'indicator_', 'n_features_in_', 'statistics_']

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
        <li>What are the attributes of a <tt>LogisticRegression</tt> classifier?. You will decompose these in the 3 categories.</li>
    </ul>
</div>

In [26]:
clf = LogisticRegression()
public_attributes = [attr for attr in dir(clf) if not attr.startswith('_')]
public_attributes

['C',
 'class_weight',
 'decision_function',
 'densify',
 'dual',
 'fit',
 'fit_intercept',
 'get_metadata_routing',
 'get_params',
 'intercept_scaling',
 'l1_ratio',
 'max_iter',
 'multi_class',
 'n_jobs',
 'penalty',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'random_state',
 'score',
 'set_fit_request',
 'set_params',
 'set_score_request',
 'solver',
 'sparsify',
 'tol',
 'verbose',
 'warm_start']

### Using a Pipeline

We saw earlier that we should be careful when preprocessing data to avoid any "data leak" (i.e. reusing some knowledge from the training when testing our model). Scikit-learn provides the `Pipeline` class to make successive transformations. In addition, it will ensure that the right operations will be applied at the right time.

In [27]:
from sklearn import set_config

set_config(display='diagram')

In [28]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(SimpleImputer(strategy='mean'),
                      LogisticRegression())
model.fit(X_train_num, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('logisticregression', LogisticRegression())])

Alternative syntax using named "steps".

In [236]:
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("imputer", SimpleImputer(strategy='mean')),
    ("classifier", LogisticRegression())    
])
model.fit(X_train_num, y_train)

Pipeline(steps=[('imputer', SimpleImputer()),
                ('classifier', LogisticRegression())])

In [238]:
model.score(X_test_num, y_test)

0.5609756097560976

Saving your estimator in HTML for presentations, blog posts etc.

In [239]:
from sklearn.utils import estimator_html_repr

with open('model.html', 'w') as fid:
    fid.write(estimator_html_repr(model))

# !open model.html

### Manipulating Pipeline steps

A pipeline is a sequence of `steps`. Each `step` is a scikit-learn estimator. All steps except the last one are typically **transformers** (i.e they have `fit`, `fit_transform`, `transform` methods) and the last step is a **classifier** or a **regressor**.

In [32]:
model.steps  # accessing steps as a list

[('imputer', SimpleImputer()), ('classifier', LogisticRegression())]

In [33]:
model.named_steps  # accessing steps with their names as a dict

{'imputer': SimpleImputer(), 'classifier': LogisticRegression()}

In [240]:
model.named_steps["imputer"]        # access the imputer step in the pipeline

SimpleImputer()

In [34]:
model[:1]  # slicing a pipeline

Pipeline(steps=[('imputer', SimpleImputer())])

In [35]:
model[-1]

LogisticRegression()

In [36]:
from sklearn.base import is_classifier

is_classifier(model[-1])

True

For comparison, we can decompose the pipeline and chain the operations manually, mimicking what the `Pipeline` object does internally:

In [37]:
preprocessor = model[:-1]
classifier = model[-1]

X_train_preproc = preprocessor.fit_transform(X_train_num, y_train) # fit_transform is used for training data
X_test_preproc = preprocessor.transform(X_test_num)                # transform is used for test data

classifier.fit(X_train_preproc, y_train)
classifier.score(X_test_preproc, y_test)

0.6707317073170732

### Towards a ColumnTransformer

If we want to avoid creating `X_train_num` and fit the model on `X_train`, we can select the numerical columns using a `ColumnTransformer` object:

In [38]:
from sklearn.compose import ColumnTransformer

ColumnTransformer?

Init signature:
ColumnTransformer(
    transformers,
    *,
    remainder='drop',
    sparse_threshold=0.3,
    n_jobs=None,
    transformer_weights=None,
    verbose=False,
    verbose_feature_names_out=True,
)
Docstring:     
Applies transformers to columns of an array or pandas DataFrame.

This estimator allows different columns or column subsets of the input
to be transformed separately and the features generated by each transformer
will be concatenated to form a single feature space.
This is useful for heterogeneous or columnar data, to combine several
feature extraction mechanisms or transformations into a single transformer.

Read more in the :ref:`User Guide <column_transformer>`.

.. versionadded:: 0.20

Parameters
----------
transformers : list of tuples
    List of (name, transformer, columns) tuples specifying the
    transformer objects to be applied to subsets of the data.

    name : str
        Like in Pipeline and FeatureUnion, this allows the transformer and
        i

In [39]:
numerical_preprocessing = ColumnTransformer([
    ("imputer", SimpleImputer(strategy='mean'), num_cols)
])
    
model = Pipeline([
    ("numerical preproc.", numerical_preprocessing),
    ("classifier", LogisticRegression()),
])

model.fit(X_train, y_train)

Pipeline(steps=[('numerical preproc.',
                 ColumnTransformer(transformers=[('imputer', SimpleImputer(),
                                                  ['pclass', 'age', 'parch',
                                                   'fare'])])),
                ('classifier', LogisticRegression())])

In [40]:
model.score(X_test, y_test)

0.6707317073170732

# 2 Working only with categorical data <a class="anchor" id="workingcategorical"></a> [↑](#Table-of-contents)

Categorical columns are not supported natively by machine-learning algorithms and require a preprocessing step commonly called encoding. The most classical categorical encoders are the `OrdinalEncoder` and the `OneHotEncoder`, but many others exist and are implemented in `sklearn`; see [here](https://scikit-learn.org/dev/modules/preprocessing.html#encoding-categorical-features). 

Let's see what happens if we try to classify the data using only categorical features encoded via `OrdinalEncoder`.

In [41]:
X_train.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1139,3.0,"Rekic, Mr. Tido",male,38.0,0.0,0.0,349249,7.8958,NaN,S,NaN,NaN,NaN
678,3.0,"Boulos, Master. Akar",male,6.0,1.0,1.0,2678,15.2458,NaN,C,NaN,NaN,"Syria Kent, ON"
290,1.0,"Taussig, Mr. Emil",male,52.0,1.0,1.0,110413,79.6500,E67,S,NaN,NaN,"New York, NY"
285,1.0,"Straus, Mr. Isidor",male,67.0,1.0,0.0,PC 17483,221.7792,C55 C57,S,NaN,96.0,"New York, NY"
1157,3.0,"Rosblom, Mr. Viktor Richard",male,18.0,1.0,1.0,370129,20.2125,NaN,S,NaN,NaN,NaN


In [42]:
cat_cols = ['sex', 'embarked', 'pclass']

In [45]:
X_train_cat = X_train[cat_cols]

In [46]:
X_train_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 1139 to 1126
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   sex       981 non-null    category
 1   embarked  980 non-null    category
 2   pclass    981 non-null    float64 
dtypes: category(2), float64(1)
memory usage: 17.5 KB


In [267]:
X_train_cat[X_train_cat['embarked'].isna() == True]

,sex,embarked,pclass
284,female,NaN,1.0


In [280]:
imputer_test = SimpleImputer(strategy='constant', fill_value='missing')
imputer_test.fit(X_train_cat)
cat_test = pd.DataFrame(imputer_test.transform(X_train_cat), columns=X_train_cat.columns)
cat_test.loc[283:285]

,sex,embarked,pclass
283,female,C,3.0
284,male,S,2.0
285,male,Q,3.0


In [47]:
from sklearn.preprocessing import OrdinalEncoder

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy='constant', fill_value='missing')),
    ("ordinal_encoder", OrdinalEncoder())
])

categorical_preprocessing = ColumnTransformer([
    ("categorical_preproc", cat_pipeline, cat_cols)
])

model = Pipeline([
    ("categorical_preproc", categorical_preprocessing),
    ("classifier", LogisticRegression())
])
model.fit(X_train, y_train)

Pipeline(steps=[('categorical_preproc',
                 ColumnTransformer(transformers=[('categorical_preproc',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['sex', 'embarked',
                                                   'pclass'])])),
                ('classifier', LogisticRegression())])

In [49]:
model.score(X_test, y_test)

0.7804878048780488

Remember that `Pipeline` is yet another scikit-learn estimator. It therefore has the methods `set_params` and `get_params` that can be inspected:

In [50]:
model.get_params()

{'memory': None,
 'steps': [('categorical_preproc',
   ColumnTransformer(transformers=[('categorical_preproc',
                                    Pipeline(steps=[('imputer',
                                                     SimpleImputer(fill_value='missing',
                                                                   strategy='constant')),
                                                    ('ordinal_encoder',
                                                     OrdinalEncoder())]),
                                    ['sex', 'embarked', 'pclass'])])),
  ('classifier', LogisticRegression())],
 'verbose': False,
 'categorical_preproc': ColumnTransformer(transformers=[('categorical_preproc',
                                  Pipeline(steps=[('imputer',
                                                   SimpleImputer(fill_value='missing',
                                                                 strategy='constant')),
                                                  ('

In [51]:
model.set_params(classifier__C=0.1)
model.get_params()

{'memory': None,
 'steps': [('categorical_preproc',
   ColumnTransformer(transformers=[('categorical_preproc',
                                    Pipeline(steps=[('imputer',
                                                     SimpleImputer(fill_value='missing',
                                                                   strategy='constant')),
                                                    ('ordinal_encoder',
                                                     OrdinalEncoder())]),
                                    ['sex', 'embarked', 'pclass'])])),
  ('classifier', LogisticRegression(C=0.1))],
 'verbose': False,
 'categorical_preproc': ColumnTransformer(transformers=[('categorical_preproc',
                                  Pipeline(steps=[('imputer',
                                                   SimpleImputer(fill_value='missing',
                                                                 strategy='constant')),
                                               

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    There exists, of course, many other strategies to encode categorical variables. In this exercise, you will write one by yourself that we shall call <tt>CountEncoder</tt>. The idea is to replace categorical variables with their count in the train set. When you've finished coding, change the ordinal encoder in the previous pipeline with an instance of your <tt>CountEncoder</tt> and see how the classification performance changes.
</div>

Your class will needs to inherit from `BaseEstimator`, `TransformerMixin` in `sklearn.base` submodule.

You will use the class `Counter` from the `collections` module in the Python standard library.

You will apply your code on this toy example

```python
>>> X = np.array([
...    [0, 2],
...    [1, 3],
...    [1, 1],
...    [1, 1],
... ])
>>> ce = CountEncoder()
>>> print(ce.fit_transform(X))
[[1 1]
 [3 1]
 [3 2]
 [3 2]]
```

Solution is in `solutions/01-count_encoder.py`.

In [266]:
from sklearn.base import BaseEstimator, TransformerMixin
from collections import Counter

class CountEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):   # apprendre les comptages de chaque valeur dans les colonnes du jeu de données
        self.counts_ = {}   # Réinitialise les comptages
        # itère sur chaque colonne pour compter les occurrences
        for col_idx in range(X.shape[1]):
            col_counts = Counter(X[:, col_idx]) # Compte les valeurs uniques 
            self.counts_[col_idx] = dict(col_counts)    # stock le dictionnaire des comptages pour la colonne
        return self

    def transform(self, X):     # appliquer ces comptages et remplacer les valeurs originales par leur fréquence
        X_transformed = X.copy()
        # for col_idx in range(X.shape[1]):
        #     for idx in range(X.shape[0]):
        #         X_transformed[idx, col_idx] = self.counts_[col_idx].get(X[idx, col_idx], 0)

        # return X_transformed

        # version optimisée, sans 2 boucles for
        for col_idx in range(X.shape[1]):
            X_transformed[:, col_idx] = np.vectorize(self.counts_[col_idx].get)(X[:, col_idx])
        
        return X_transformed

X = np.array([
    [0, 2],
    [1, 3],
    [1, 1],
    [1, 1]
    ])

ce = CountEncoder()
print(ce.fit_transform(X))

[[1 1]
 [3 1]
 [3 2]
 [3 2]]


# 3 Mix categorical and numerical data in a pipeline <a class="anchor" id="combining"></a> [↑](#Table-of-contents)

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
    <li>Try to combine the numerical and categorical pipelines into a single <tt>ColumnTransformer</tt></li>
        <li>Fit a <tt>LogisticRegression</tt> classifier on the output of this feature engineering. How does the test score evolve?</li>
    </ul>
</div>

Solution is in `solutions/01b-full_column_transformer.py`

In [309]:
from sklearn.preprocessing import StandardScaler

num_cols = ['pclass', 'age', 'parch', 'fare']
num_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy='mean')),    # Imputation des valeurs manquantes
        ("scaler", StandardScaler())                    # Standardisation pour mieux converger
    ]
)

cat_cols = ['sex', 'embarked', 'pclass']
cat_transformer = Pipeline([
        ("inputer", SimpleImputer(strategy='constant', fill_value='missing')),
        ("ordinal_encoder", OrdinalEncoder())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_cols),     # colonnes numériques
        ("cat", cat_transformer, cat_cols)      # colonnes catégorielles
    ]
)

both_model = Pipeline([
        ("preprocessor", preprocessor),         # Prétraitement
        ("classifier", LogisticRegression())    # Classifieur
    ]
)

both_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'parch',
                                                   'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('inputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['sex', 'embarked',
                                                   'pclass'])])),
                ('classifier', LogisticRegression())])

In [310]:
both_model.fit(X_train, y_train)    # Entraînement du modèle

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'parch',
                                                   'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('inputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['sex', 'embarked',
                                                   'pclass'])])),
                ('classifier', LogisticRegression())])

In [311]:
both_model.score(X_test, y_test)

0.7957317073170732

The procedure carried out so far was, until very recently, the standard way of handling tabular data mixing categorical and numerical values. Although this works well, it is not very straightforward and can require a great deal of expertise from the user. This was one of the main motivations behind the creation of [`skrub`](https://skrub-data.org/stable/index.html), a Python library that facilitates the preprocessing and feature engineering of tabular machine learning.

<img src="figures/skrub-logo.png" width="100">

One of the most important objects in `skrub` is the `TableVectorizer`, which automatically chooses which encoding strategy should be used for each column depending on its cardinality. See below what we get when using it as the first step in a `sklearn` pipeline:

In [312]:
from skrub import TableVectorizer

model = Pipeline([
    ("table_vectorizer", TableVectorizer()),
    ("imputer", SimpleImputer(strategy='mean')),
    ("classifier", LogisticRegression(solver='liblinear'))
])
model.fit(X_train, y_train)

Pipeline(steps=[('table_vectorizer', TableVectorizer()),
                ('imputer', SimpleImputer()),
                ('classifier', LogisticRegression(solver='liblinear'))])

In [313]:
print(model.score(X_test, y_test))

/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


0.9695121951219512


# 4 From one split to cross-validation <a class="anchor" id="crossvalidation"></a> [↑](#Table-of-contents)

Cross-validation objects are parametrized to split data in multiple train and test splits as described [here](https://scikit-learn.org/1.5/modules/cross_validation.html).

These objects are called **splitters** and consist of

- A `split` method that generate indices to partition data into training and test set.
- A `get_n_splits` method that returns the number of splitting iterations in the cross-validator

See [this](https://github.com/scikit-learn/scikit-learn/blob/6e9039160f0dfc3153643143af4cfdca941d2045/sklearn/model_selection/_split.py#L109) chunk of code for the structure of a basic splitter.

Given a `model`, some data `X, y` and a splitter one can fit and score on
all requested data splits. Some functions to do this automatically are `cross_val_score`
(historical way) and `cross_validate` (more modern way).

Let's first define a model:

In [61]:
from sklearn.preprocessing import OrdinalEncoder

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ordinal_encoder", OrdinalEncoder())
])

categorical_preprocessing = ColumnTransformer([
    ("categorical_preproc", cat_pipeline, cat_cols)
])

model = Pipeline([
    ("categorical_preproc", categorical_preprocessing),
    ("classifier", LogisticRegression())
])
model

Pipeline(steps=[('categorical_preproc',
                 ColumnTransformer(transformers=[('categorical_preproc',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinal_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['sex', 'embarked',
                                                   'pclass'])])),
                ('classifier', LogisticRegression())])

Let's now use a default 5-fold CV. You will see that there is a large amount of discrepancy among the test_score values

In [314]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, X_df, y, cv=5)
pd.DataFrame(cv_results)


/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/paulinezhou/opt/anaco

,fit_time,score_time,test_score
0,4.058707,1.582516,0.862595
1,4.342851,2.195660,0.973282
2,4.807613,2.097458,0.885496
3,6.227730,1.674008,0.927481
4,4.909904,2.043781,0.931034


In [63]:
pd.DataFrame(cv_results).agg(['mean', 'std'])

,fit_time,score_time,test_score
mean,0.024093,0.005662,0.711246
std,0.013657,0.001216,0.121393


The reason is that default CV object (here 5-fold CV) is deterministic, while the distribution of "survivor" is not uniform in the dataset. See:

In [337]:
y_grouped_mean = y.astype(int).to_frame().groupby(y.index.values // 100).mean()           # astype de pandas permet de changer le type de données (dtype) d'une colonne ou d'un objet entier (Série ou df)
y_grouped_mean

,survived
0,0.670000
1,0.640000
2,0.580000
3,0.460000
4,0.340000
5,0.500000
6,0.270000
7,0.260000
8,0.250000
9,0.260000


In [346]:
y_grouped_mean.plot(y="survived", legend = False, title = "moyenne par groupe")
plt.xlabel("Group")


Text(0.5, 0, 'Group')

Error in callback <function _draw_all_if_interactive at 0x142a9df70> (for post_execute):


ValueError: object __array__ method not producing an array

ValueError: object __array__ method not producing an array

<Figure size 640x480 with 1 Axes>

To fix this one needs stratified folds (so that the fraction of "survivors" is the same in each fold) but also to shuffle the data.

In [341]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_validate(model, X_df, y, cv=cv)
pd.DataFrame(cv_results)

/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/paulinezhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_score
0,4.745304,1.731889,0.969466
1,5.162510,1.724608,0.980916
2,5.363721,1.823926,0.973282
3,4.504091,1.901117,0.969466
4,4.626075,1.815981,0.957854


In [66]:
pd.DataFrame(cv_results).agg(['mean', 'std'])

,fit_time,score_time,test_score
mean,0.018032,0.006970,0.771595
std,0.005327,0.005084,0.025519


The variance across folds is now much smaller which is great!

Let's look at the cross-validation scheme with a pretty plot:

In [342]:
from plotting_utils import plot_cv_indices
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(9, 4))

for shuffle, ax in zip([False, True], axes):
    cv = StratifiedKFold(n_splits=5, shuffle=shuffle)
    plot_cv_indices(cv, X_df, y, ax=ax)

fig.tight_layout()

/Users/paulinezhou/Desktop/ENSTA/3A/DataCamp/datacamp-master/02_pipelines_and_column_transformers/plotting_utils.py:37: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(
/Users/paulinezhou/Desktop/ENSTA/3A/DataCamp/datacamp-master/02_pipelines_and_column_transformers/plotting_utils.py:37: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(


Error in callback <function _draw_all_if_interactive at 0x142a9df70> (for post_execute):


ValueError: object __array__ method not producing an array

ValueError: object __array__ method not producing an array

<Figure size 900x400 with 2 Axes>

See [this page](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html) for more details and a list of the different cross-validation objects.

### Writing your own cross-validation object

As mentioned above, every splitter should implement a `split` and `get_n_splits` method. 

- `split` should return an iterable of tuples of indices. This is isually done using the concept of `yield` in Python for generators.
- `get_n_splits` returns an integer corresponding to the number of splits/folds chosen at the initialization of the splitter.

Let's first see what we get with the `cv` object defined before.

In [68]:
cv.get_n_splits()

5

In [69]:
for train_idx, test_idx in cv.split(X_df, y):
    print(train_idx[:5], test_idx[:5])
    print(f"N. samples train: {len(train_idx)}  -- N. samples test: {len(test_idx)}")

[1 2 4 5 6] [ 0  3  7 12 13]
N. samples train: 1047  -- N. samples test: 262
[0 1 2 3 4] [ 6 10 25 38 40]
N. samples train: 1047  -- N. samples test: 262
[0 3 6 7 9] [1 2 4 5 8]
N. samples train: 1047  -- N. samples test: 262
[0 1 2 3 4] [ 9 11 15 16 17]
N. samples train: 1047  -- N. samples test: 262
[0 1 2 3 4] [18 26 39 41 44]
N. samples train: 1048  -- N. samples test: 261


<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
        <li>Imagine that the index of <code>y</code> gives you some provenance about the sample (e.g. which cohort of subjects in a clinical study). Write a splitter that allows to test the performance of a model on a left-out cohort. In other words, you will do as many splits as the number of unique values in <code>y.index.values</code>, and predict of each left-out cohort. To simulate this, we will modify the index variable <code>y</code>, just for educational purposes.
        </li>
    </ul>
</div>

Solution is in `solutions/01c-splitter.py`

In [345]:
from sklearn.model_selection import BaseCrossValidator

class MySplitter(BaseCrossValidator):

    def __init__(self):
        pass

    def get_n_splits(self, X=None, y=None, groups=None):
        return len(np.unique(y.index.values))

    def split(self, X, y, groups=None):
        splits_idx = np.unique(y.index.values)
        idx = np.arange(len(X))
        for k in splits_idx:
            mask = (y.index.values == k)
            train_idx = idx[~mask]
            test_idx = idx[mask]
            yield train_idx, test_idx
        

cv = MySplitter()
y_with_provenance = y.copy()
plot_cv_indices(cv, X_df, y_with_provenance)


/Users/paulinezhou/Desktop/ENSTA/3A/DataCamp/datacamp-master/02_pipelines_and_column_transformers/plotting_utils.py:37: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(


Error in callback <function _draw_all_if_interactive at 0x142a9df70> (for post_execute):


ValueError: object __array__ method not producing an array

ValueError: object __array__ method not producing an array

<Figure size 640x480 with 1 Axes>

In [70]:
y_with_provenance = y.copy()
y_with_provenance.index = y_with_provenance.index.values // 200  # to easily mimic 5 cohorts
n_splits = y_with_provenance.index.nunique()
print(n_splits)

7


### When you're done with this notebook you can do the assignments on scikit-learn.